# Explaining microbial scaling laws using Bayesian inference
In this project, we want to combine methods from _Statistical Physics_ and _Bayesian Data Analysis_ to elucidate the principles behind cellular growth and division. We will study various classes of individual-based growth-division models and infer individal-level processes (model structures and likely ranges of associated parameters) from sigle-cell observations. 

In the _Bayesian framework_, we formalize our process understanding the form of different rate functions, expressing the dependence of growth and division rates on variables characterizing a cell’s state (such as size and protein content), and calculate the Bayesian posteriors for the parameters of these functions.

## Group 
- [Tommaso Amico](https://github.com/tommasoamico)
- [Andrea Lazzari](https://github.com/AndreaLazzari)
- [Paolo Zinesi](https://github.com/PaoloZinesi)
- [Nicola Zomer](https://github.com/NicolaZomer)

## Overview
### Growth and division processes: general model 
In our models we consider the evolution of a single non-interacting cell, which undergoes 2 processes:
- **growth:** the cell size $x(t)$ evolves according to the following equation
    $$
    \dot{x}=g(x(t)) \quad, \quad x(0)=x_b
    $$
    In some cases this relation can be expressed in vectorial form, where $\underline{x}$ is the vector of the traits characterizing the cell's state (see model 2). 
- **division:** it is ruled by the _hazard rate function_ $h(x(t))$, which represents the istantaneous probability of the cell to divide. This function is related to the so called _survival function_ $s(t)$, by the relation
    $$
    \frac{\dot{s}(t)}{s(t)}=-h(t) \quad , \quad s(0)=1
    $$
    where $s(t)$ gives the probability that the cell will survive (meaning not divide in this case) past a certain time $t$.

While the growth is a deterministic process, division is a stochastic event. Since the cell does not generally divide into two equal parts, we introduce a parameter $frac$, which is treated as a stochastic variable, such that after the division
$$
\underline{x}_{div} = \left(frac\cdot x, (1-frac)\cdot x\right)
$$

Finally, we assume that the division ratios $frac$ are distributed according to a Beta function and that the growth rates $\omega_1$ follow a Gamma function, hence denoting by $f$ the probability density distribution we obtain
$$
\begin{aligned}
f(frac|a, b) &= Beta(a, b) \\
f(\omega_1|c, d) &= Gamma(c, d)
\end{aligned}
$$


**Workflow**
- _Calibration_: <br> 
  Performing Markov Chain Monte Carlo (MCMC), via the Python implementation [emcee](https://emcee.readthedocs.io/en/stable/), we find the posterior distribution of $\theta$ and the marginalized posterior of each parameter, of which we calculate the maximum, the median and the 95% credibility interval. Then, we use this results to generate a simulated time series, that can be compared with the real data, to find which model is statistically better.  
- _Validation_: <br>
  Model validation and comparison is achieved by 
  - making a boxplot of the simulated and real interdivision times
  - computing the overlap of the histograms of the interdivision times
  - calculating the predictive density


## Methods
+ **Sampling from the CDF distribution with numerical inversion**
Following the inverse transform method, to draw a division time $t_d$ from the associated distribution $s(t)$, we draw a random value $K$ from a uniform distribution in the interval $[0, 1]$ (i.e., we draw a "survival probability”) and solve numerically for $t_d$

$$ 
\log(s(t_d)) = \log(K)
$$


We use the library [Pyinverse](https://pypi.org/project/pynverse/) to find numerically the inverse of a function.

+ **Forward propagation**
At this point, given the time series of the survival times, we can forward propagate the evolution of a single cell, according to the following procedure:

1. Start with the initial size $x_b$
2. Propagate using the equation for $x(t)$ up to the first division time
3. Divide the cell size by 2 (assuming the cell divides equally in two parts) INITIAL ASSUMP ON f, THEN WITH f
4. Repeat the last 2 points for all division times generated, imposing as initial size of the cell the one after division and resetting the time

Recall that in this model the cell growth is a deterministic process, while the division is stochastic.



### Model 0 - Linear
We start with a very simple stochastic model, biologically not very realistic, but useful to start familiarizing with the problem. In this first model we define $g(x)$ and $h(x)$ as 2 linear functions
$$
\begin{aligned}
g(x) &\equiv \omega_1(\mu+x) \\
h(x) &\equiv \omega_2(1+x/\nu)
\end{aligned}
$$
where $\omega_1$ and $\omega_2$ are frequencies, while $\mu$ and $\nu$ are sizes (tipycally measured in $\mu m$). The ratio between $\omega_1$ and $\omega_2$ is the order parameter that triggers the phase transition. The parameters $\mu$ and $\nu$ are necessary to cut off the probability distribution (in zero and for large values of $x$), which is important both for physical reasons and for making the distribution normalizable. Introducing these parameters is a mathematical trick, useful for example to prevent the cell from having a too small size, which however is difficult to justify from a biological point of view. We will then see better models, biologically speaking.

The results of the two differential equations are:
$$
\begin{aligned}
x(t) &= \left(x_b+\frac{\mu}{\omega_1}\right)e^{\omega_1 t}-\frac{\mu}{\omega_1} \\
\ln(s(t)) &= \omega_2 t \left(\frac{u}{\nu}-1\right) + \frac{\omega_2}{\omega_1} \left(\frac{u+x_b}{\nu}\right) (1-e^{\omega_1 t}) \\
s(t) &= \exp\left\{ \omega_2t \left(\frac{u}{\nu}-1\right) + \frac{\omega_2}{\omega_1}\left(\frac{u + x_b}{\nu}\right)(1-e^{\omega_1t})\right\}
\end{aligned}
$$


![img](images/s_model0.png)

### Model 1 - with Plateau
As in the previous model, even in this case the cell growth is governed by a single trait, which is the size. However, this model is biologically more realistic, mainly because a lower bound is placed on the size of the cell such that it can divide. 

Also in this case the processes considered are growth and division, governed by $g(x)$ and $h(x)$ respectively. In this model we define $g(x)$ and $h(x)$ as follows
$$
\begin{aligned}
g(x)&= \omega_1 x \\
h(x)&=
\begin{cases}
    0  & , \, x<\mu \\
    \omega_2 \cdot \frac{x+v}{u+v} & , \, x\geq \mu
\end{cases}
\end{aligned}
$$
where $g(x)$ again corresponds to an exponential growth, while $h(x)$ is lower bounded by $u$.

The results of two *differential equations* are:
$$
\begin{aligned}

x(t) &= x_b\, e^{\omega_1 t} \\

\ln(s(t)) &= - \left[ \dfrac{x_b}{\mu+\nu} \dfrac{\omega_2}{\omega_1} \left( e^{\omega_1 t} - e^{\omega_1 t_0}\right) + \dfrac{\nu}{\mu+\nu}\, \omega_2 (t-t_0) \right] \theta(t-t_0) \\

s(t) &= \exp\left\{ - \left[ \dfrac{x_b}{\mu+\nu} \dfrac{\omega_2}{\omega_1} \left( e^{\omega_1 t} - e^{\omega_1 t_0}\right) + \dfrac{\nu}{\mu+\nu}\, \omega_2 (t-t_0) \right] \theta(t-t_0) \right\}


\end{aligned}
$$

where $t_0 \equiv \max\left\{0,\dfrac{1}{\omega_1} \ln\left(\dfrac{\mu}{x_b}\right)\right\}$ is the minimum time at which the cell can divide and $\theta(\tau)$ is the Heaviside function.

Note that this model is different from the linear one only if the new parameter $t_0$ is bigger than zero, i.e., only if $\mu > x_b$.


![img](images/s_model1.png)


### Model 2 - with Protein
The main difference between this model and the previous ones is that here we consider 2 traits: the cell size $m(t)$ and its protein content $p(t)$. We call $\underline{x}$ the vector
$$
\underline{x} = \begin{pmatrix} m\\ p\end{pmatrix} 
$$

As before, the traits evolution and the cell division are governed by $g(\underline{x})$ and $h(p)$ respectively, which are defined as 
$$
\begin{aligned}
g(\underline{x})&=\omega_1m\begin{pmatrix} 1\\ c\end{pmatrix} \\
h(p)&=
    \begin{cases}
    0   & , \, p<\mu \\
    \omega_2 \, \frac{p+v}{u+v} & , \, p\geq \mu
    \end{cases}
\end{aligned}
$$
From $g(\underline{x})$ we can notice that the cell size still grows exponentially and the protein content also follows this evolution, scaled by the factor $c$. As $c$ doesn't have a real meaning, we set it to $1$. 

Moreover, in this model the condition under which the cell can divide is that it contains a minimum amount of a specific type of protein, which we call $u$. If $p\geq u$ the cell can divide, otherwise it cannot. Unlike model 1, we do not have any condition on the size of the cell for the division to take place and $h$ depens only on $p$.

The initial conditions for $m(t)$ and $p(t)$ are
$$
\begin{aligned}
    p(t=0) &= 0 \\
    m(t=0) &= m_b
\end{aligned}
$$

and the division process occurs in the following way
$$
\begin{pmatrix} m \\ p\end{pmatrix} \rightarrow \begin{pmatrix} frac \cdot m \\ 0\end{pmatrix} + \begin{pmatrix} (1-frac) \cdot m \\ 0\end{pmatrix}
$$
where $frac$ is the division ratio.

Similarly to the [Model 1](/analysis_real_data/REAL_Model_1.ipynb), the results of the two differential equations are:
$$
\begin{aligned}

m(t) &= m_b\, e^{\omega_1 t} \\



\ln(s(t)) &= - \left[ \dfrac{m_b}{\mu+\nu} \dfrac{\omega_2}{\omega_1} \left( e^{\omega_1 t} - e^{\omega_1 t_0}\right) + \dfrac{\nu - m_b}{\mu+\nu}\, \omega_2 (t-t_0) \right] \theta   (t-t_0) \\




%s(t) &= \exp\left\{ - \left[ \dfrac{m_b}{\mu+\nu} \dfrac{\omega_2}{\omega_1} \left.         %(e^\omega_1 t} - e^{\omega_1 t_0}\right) + \dfrac{\nu - m_b}{\mu+\nu}\, \omega_2 (t-t_0) %\right] \theta(t-t_0) \right\}



\end{aligned}

$$

where $t_0 \equiv \dfrac{1}{\omega_1} \ln\left(1 + \dfrac{\mu}{m_b}\right) $ is the minimum time at which the cell can divide and $\theta(\tau)$ is the Heaviside function.

Note that in this model $t_0$ is always bigger than zero, because the argument of the logarithm is alway bigger than one. This means that, independently on the choice of parameters, each cell has a time period in which it cannot divide because the protein has not reached yet the threshold value $\mu$.

![img](images/s_model2.png)

### Bayesian Data Analysis
For all models, the set of parameters to be inferred is 
$$
\underline{\theta} = \{\mu, \nu, \omega_2, a, b, c, d\}
$$

Applying the Bayes theorem, we can write
$$
f(\underline{\theta}|\tau, \omega_1, frac, M) \propto f(\tau, \omega_1, frac|\underline{\theta}, M)\cdot f(\underline{\theta}, M)
$$
where $M$ is the background information given by the selected model and $\tau$, $\omega_1$ and $frac$ are provided by the data.

Regarding the likelihood, $f(\tau, \omega_1, frac|\underline{\theta})$, assuming $\tau$, $\omega_1$ and $frac$ are independent, it can be written as the product of the probability density function of each of them 
$$
f(\tau, \omega_1, frac|\underline{\theta}) = 
f(\tau|\omega_1, frac, \underline{\theta}) \cdot f(\omega_1|\underline{\theta}) \cdot f(frac|\underline{\theta})
$$ 
where the last 2 are respectively the $Gamma(c, d)$ and $Beta(a, b)$ distributions, while the former is the probability density function of division times, which depends on the selected model and it is the derivative of the survival function $s(t)$.

To rank the different models, we now calculate the predictive density of each model. 

First we divide the dataset in 2 parts, one used for calibration and the other for validation. This is done randomly and several times, in order to get a more robust estimate of the predictive density. 

In the calibration part, we take the maximum a posteriori (MAP) of the posterior: $\underline{\theta}_{MAP}$. Then, at each iteration, we use the inferred values of the parameters and the validation set to calculate the predictive density: 
$$
    \mathcal{L}(\underline{\tau}^V, \underline{\omega}_1^V,\underline{f}^V | \underline{\theta}_{MAP}) =
    \prod_{cells} 
    \mathcal{L}(\underline{\tau}_i^V | \underline{\omega}_{1, i}^V,\underline{f}_i^V,\underline{\theta}_{MAP}) 
    \cdot \mathcal{L}(\underline{\omega}_{1, i}^V | \underline{\theta}_{MAP}) 
    \cdot \mathcal{L}(\underline{f}_i^V | \underline{\theta}_{MAP})
$$

Finally, we take the average of the values obtained at each iteration for each model. Given the 3 models, the best is the one with the largest value of $\mathcal{L}(\underline{\tau}^V, \underline{\omega}_1^V,\underline{f}^V | \underline{\theta}_{MAP})$.

TO DO LIST - 

+ <s>PLOT s(t) for each model, just for the intro</s>
+ <s>Add analytical solutions in the presentation part, before showing the notebooks</s>
+ Add theory of predictive density

## First Simulations

+ why start from here
+ hyperlinks to all the simulated models

Fast presentations of the notebooks?


+ [Starting Model](/analysis_sim_data/starting_model.ipynb)
+ [Model 1](/analysis_sim_data/model_1.ipynb)
+ [Model 2](/analysis_sim_data/model_2.ipynb)

## Real Data

+ Dataframe of real data
+ 3D plot to show the data

+ hyperlinks to each notebook
   some comments, all the inference plots


+ [Starting Model](/analysis_real_data/REAL_starting_model.ipynb)
+ [Model 1](/analysis_real_data/REAL_Model_1.ipynb)
+ [Model 2](/analysis_real_data/REAL_Model_2.ipynb)


First, we import the real data and we create the following objects:
- `size`: length of the time series, i.e., the number of divisions 
- `xb_realdata`: initial size of the cell 
- `df_frac`: series of division ratios
- `df_growth`: series of division rates 
- `df_times`: series of inter-division times

In [22]:
import pandas as pd
import numpy as np


df_microbial_growth = pd.read_csv('/work/Microbial_Scaling_Laws/data/modified_Susman18_physical_units.csv')
xb_realdata = df_microbial_growth['length_birth'][0]

# remove first NaN
df_microbial_growth  =  df_microbial_growth[~ np.isnan(df_microbial_growth['division_ratio'])]

# check that there is only one cell in the dataset
print('Number of cells in the dataset:', df_microbial_growth['lineage_ID'].nunique()) 

display(df_microbial_growth)


Number of cells in the dataset: 1


,div_and_fold,fold_growth,division_ratio,added_length,generationtime,length_birth,length_final,growth_rate,lineage_ID,generation,start_time,end_time
1,1.009831,0.794984,0.456028,3.198324,0.500000,2.633655,5.831979,1.589968,15.0,1.0,1.083333,1.583333
2,0.862482,0.607893,0.469619,2.291172,0.416667,2.738808,5.029980,1.458944,15.0,2.0,1.666667,2.083333
3,1.426486,1.037370,0.505526,4.632409,0.833333,2.542786,7.175195,1.244844,15.0,3.0,2.166667,3.000000
4,0.729601,0.490156,0.446903,2.028413,0.333333,3.206619,5.235033,1.470468,15.0,4.0,3.083333,3.416667
5,0.887982,0.609464,0.482744,2.121432,0.416667,2.527183,4.648614,1.462714,15.0,5.0,3.500000,3.916667
...,...,...,...,...,...,...,...,...,...,...,...,...
247,0.940427,0.590789,0.520893,2.192641,0.416667,2.722385,4.915026,1.417893,15.0,249.0,150.583333,151.000000
248,0.928602,0.604227,0.507478,2.069835,0.500000,2.494268,4.564104,1.208453,15.0,250.0,151.083333,151.583333
249,1.568073,1.046815,0.550478,4.644408,0.750000,2.512440,7.156848,1.395754,15.0,251.0,151.666667,152.416667
250,0.809727,0.481681,0.500204,2.215210,0.333333,3.579885,5.795095,1.445043,15.0,252.0,152.500000,152.833333


In [21]:
import sys
sys.path.insert(0, '/work/Microbial_Scaling_Laws/analysis_real_data')

from Fernando_package import functions
from Fernando_package import plot_funcs

fig = plot_funcs.plot_3d_interactive(df_microbial_growth)
fig.show()

## Final Results


+ Boxplot of the simulated times

+ Overalp for the various models

## References
[1] Held J, Lorimer T, Pomati F, Stoop R, Albert C. Second-order phase transition in phytoplankton trait dynamics. _Chaos_. 2020; 30(5):053109. https://doi.org/10.1063/1.5141755 

[2] Zheng, H., Bai, Y., Jiang, M. et al. General quantitative relations linking cell growth and the cell cycle in Escherichia coli. _Nature Microbiology_. 2020;  5(8):995–1001. https://doi.org/10.1038/s41564-020-0717-x 

[3] emcee documentation: https://emcee.readthedocs.io/en/stable/

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=52dba48d-3798-4665-95fc-01a96804955b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>